# Mapping Data with Folium

![](images/folium.png)

Below, we will go through a brief introduction to the **Folium** library.  This is a nice way to build interactive visuzlizations.  We will be executing these in the jupyter notebooks, however they are easily output as `.html` files ready to be served.  To begin, let's make sure we have folium installed.  

In [1]:
import folium
import pandas as pd

We can make basic maps centered at any geolocation.  For example, below we create a basic map of Portland, Oregon.  

In [2]:
m = folium.Map(location=[45.5236, -122.6750])

In [3]:
m

In [4]:
m.save('index_map.html')

We can add arguments that include changing the style of the map and the initial zoom level.

In [5]:
folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

We can use the `popup` argument to include information to be displayed at specified marker locations.

In [6]:
tooltip = 'Click me!'
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)



folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>').add_to(m)
folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>').add_to(m)
m

We can even include `markdown` syntax and icons.

In [7]:
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

In [8]:
m

We can manually control radii for markers of interest.  Below, we plot two circles at specific locations.

In [9]:
m = folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


In [10]:
m

### Mapping Bike Data

Now, we will use a dataset from NYC's citibike data.  Our goal is to compare incoming and outgoing traffic at given stations depending on the time of day.  

In [19]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")
marker = folium.CircleMarker(location=[40.738, -73.98])
marker.add_to(folium_map)

In [20]:
folium_map

In [21]:
bikes = pd.read_csv('data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2017_release_.csv')

In [23]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27204 entries, 0 to 27203
Data columns (total 63 columns):
StateAbbr                 27204 non-null object
PlaceName                 27204 non-null object
PlaceFIPS                 27204 non-null int64
TractFIPS                 27204 non-null int64
Place_TractID             27204 non-null object
Population2010            27204 non-null int64
ACCESS2_CrudePrev         27203 non-null float64
ACCESS2_Crude95CI         27203 non-null object
ARTHRITIS_CrudePrev       27204 non-null float64
ARTHRITIS_Crude95CI       27204 non-null object
BINGE_CrudePrev           27204 non-null float64
BINGE_Crude95CI           27204 non-null object
BPHIGH_CrudePrev          27204 non-null float64
BPHIGH_Crude95CI          27204 non-null object
BPMED_CrudePrev           27204 non-null float64
BPMED_Crude95CI           27204 non-null object
CANCER_CrudePrev          27204 non-null float64
CANCER_Crude95CI          27204 non-null object
CASTHMA_CrudePrev       

In [27]:
bikes.Geolocation[0][1:6]

'33.57'

In [28]:
bikes.Geolocation[0][17:23]

'-86.72'

In [ ]:
lat = bikes.Gerlocation[0][1:6]

In [ ]:
long = 

In [18]:
bikes['starttime'] = pd.to_datetime(bikes['starttime'])
bikes['stoptime'] = pd.to_datetime(bikes['stoptime'])
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
bikes['ehour'] = bikes['stoptime'].map(lambda x: x.hour)

KeyError: 'starttime'

In [ ]:
bikes.head()

In [ ]:
locations = bikes.groupby('start station id').first()

In [ ]:
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]

In [ ]:
subset = bikes[bikes["hour"]==10]

In [ ]:
dept_counts = subset.groupby("start station id").count()

In [ ]:
dept_counts = dept_counts.iloc[:, [0]]

In [ ]:
dept_counts.columns = ["Departure Counts"]

### Problem

Repeat the above for arrivals, in anticipation of joining the two for our map.

In [ ]:
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]
subset = bikes[bikes["hour"]==18]
dept_counts = subset.groupby("start station id").count()
dept_counts = dept_counts.iloc[:, [0]]
dept_counts.columns = ["Departure Counts"]


locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["ehour"]==18]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]

In [ ]:
trip_counts = dept_counts.join(locations).join(arr_counts)

In [ ]:
trip_counts.head()

In [ ]:
for index, row in trip_counts.iterrows():
    
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    
    radius = net_departures/7
    
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)

In [ ]:
folium_map

In [ ]:
popup_text = """{}<br>
                total departures: {}<br> 
                total arrivals: {}<br>
                net departures: {}"""


popup_text = popup_text.format(row["start station name"],
                               row["Arrival Counts"],
                               row["Departure Counts"],
                               net_departures)

In [ ]:
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    radius = net_departures/7
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True, popup = popup_text).add_to(folium_map)

In [ ]:
folium_map

### PROBLEM

Compare this image to that of when people are leaving work.  Doe you see what you expect?  What does this tell you about movement in the city?

In [ ]:
locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["hour"]==10]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]